In [ ]:
%pip install chess
%pip installl tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
ERROR: unknown command "installl" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [2]:
import chess
import chess.engine
import random
import numpy
import tensorflow  # type: ignore

In [3]:
def random_board(max_depth=200):
  board = chess.Board()
  depth = random.randrange(0, max_depth)

  for _ in range(depth):
    all_moves = list(board.legal_moves)
    random_move = random.choice(all_moves)
    board.push(random_move)
    if board.is_game_over():
      break

  return board


# this function will create our f(x) (score)
def stockfish(board, depth):
  with chess.engine.SimpleEngine.popen_uci('./stockfish') as sf:
    result = sf.analyse(board, chess.engine.Limit(depth=depth))
    score = result['score'].white().score()
    return score

In [4]:
board = random_board()
board
stockfish(board, 10)

-235

In [5]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [15]:
# Function to generate random chess boards and evaluations
def generate_data(num_samples=1000):
    boards = []
    evaluations = []

    for _ in range(num_samples):
        board = random_board()
        # // normalise this between -1 and 1
        evaluation = stockfish(board, 10) / 1000.0
        boards.append(split_dims(board))
        evaluations.append(evaluation)

    return numpy.array(boards), numpy.array(evaluations)

# Function to save the dataset into an .npz file
def save_dataset(filename, boards, evaluations):
    numpy.savez(filename, b=boards, v=evaluations, allow_pickle=True)

# Generate data
x_train, y_train = generate_data(num_samples=1000)

# Save the dataset into an .numpyz file
save_dataset('random_dataset.npz', x_train, y_train)

print("Dataset saved successfully.")

Dataset saved successfully.


In [6]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

In [7]:
def build_model_residual(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(board3d)
  for _ in range(conv_depth):
    previous = x
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, previous])
    x = layers.Activation('relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

In [8]:
model = build_model(32, 4)

In [9]:
import tensorflow.keras.callbacks as callbacks

def get_dataset():
    container = numpy.load('random_dataset.npz', allow_pickle=True)
    b, v = container.get('b'), container.get('v')
    v = numpy.array(v)  # convert v to a numpy array if it's not already one
    v = numpy.where(v == None, 0, v)
    v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
    return b, v


x_train, y_train = get_dataset()
x_train.transpose()
print(x_train.shape)
print(y_train.shape)

(1000, 14, 8, 8)
(1000,)


In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()
checkpoint_filepath = '/tmp/checkpoint/model.keras'
model_checkpointing_callback = ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_best_only= True,
)
model.fit(x_train, y_train,
          batch_size=2048,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4),model_checkpointing_callback])

model.save('model.h5')

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 14, 8, 8)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 14, 8, 32)      │         2,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 8, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 8, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 8, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       229,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 259,585 (1014.00 KB)

 Trainable params: 259,585 (1014.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 849ms/step - loss: 0.0088 - val_loss: 0.0140 - learning_rate: 5.0000e-04
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 0.0086 - val_loss: 0.0137 - learning_rate: 5.0000e-04
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.0083 - val_loss: 0.0136 - learning_rate: 5.0000e-04
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.0082 - val_loss: 0.0134 - learning_rate: 5.0000e-04
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 0.0079 - val_loss: 0.0132 - learning_rate: 5.0000e-04
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.0076 - val_loss: 0.0129 - learning_rate: 5.0000e-04
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.0073 - val_loss: 0.0126 - learning_rate: 5.0000e-04
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.0069 - val_loss: 0.0123 - learning_rate: 5.0000e-04
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.0066 - val_loss: 0.0121 - 

In [11]:
from tensorflow.keras import models
model = models.load_model('model.h5')

In [12]:
# used for the minimax algorithm
def minimax_eval(board):
  board3d = split_dims(board)
  board3d = numpy.expand_dims(board3d, 0)
  return model(board3d)[0][0]


def minimax(board, depth, alpha, beta, maximizing_player):
  if depth == 0 or board.is_game_over():
    return minimax_eval(board)

  if maximizing_player:
    max_eval = -numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, False)
      board.pop()
      max_eval = max(max_eval, eval)
      alpha = max(alpha, eval)
      if beta <= alpha:
        break
    return max_eval
  else:
    min_eval = numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, True)
      board.pop()
      min_eval = min(min_eval, eval)
      beta = min(beta, eval)
      if beta <= alpha:
        break
    return min_eval


# this is the actual function that gets the move from the neural network
def get_ai_move(board, depth):
  max_move = None
  max_eval = -numpy.inf

  for move in board.legal_moves:
    board.push(move)
    eval = minimax(board, depth - 1, -numpy.inf, numpy.inf, False)
    board.pop()
    if eval > max_eval:
      max_eval = eval
      max_move = move

  return max_move

In [14]:
# Testing code AI(white) vs Stockfish(black)
board = chess.Board()
from IPython.display import clear_output

with chess.engine.SimpleEngine.popen_uci('./stockfish') as engine:
    while True:
        clear_output(wait=True)
        move = get_ai_move(board, 1)
        board.push(move)
        print(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break
        move = engine.analyse(board, chess.engine.Limit(time=0.1), info=chess.engine.INFO_PV)['pv'][0]
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break

h2h4

. . k r . . r .
p p p . p p . p
. . n . . . p .
. . . . . . . .
. P . . . P . P
. . P . n . P .
P B . . . . . .
R . K . . . . .

. . k . . . r .
p p p . p p . p
. . n . . . p .
. . . . . . . .
. P . . . P . P
. . P . n . P .
P B . . . . . .
R . K r . . . .
game_over


In [1]:
# Move by move testing code AI(white) vs Stockfish(black)
import os
import chess.engine

# Get the absolute path to the stockfish executable
stockfish_path = os.path.abspath('./stockfish')

board = chess.Board()

with chess.engine.SimpleEngine.popen_uci(stockfish_path) as engine:
    while True:
        move = get_ai_move(board, 1)
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break
        move = engine.analyse(board, chess.engine.Limit(time=1), info=chess.engine.INFO_PV)['pv'][0]
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break

NameError: name 'get_ai_move' is not defined

In [58]:
board = chess.Board()
from IPython.display import clear_output

with chess.engine.SimpleEngine.popen_uci('./stockfish') as engine:
    while True:
        clear_output(wait=True)
        move = get_ai_move(board, 1)
        board.push(move)
        print(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break
        input_var = input()
        move = chess.Move.from_uci(input_var)
        board.push(move)
        print(move)
        print(f'\n{board}')
        if board.is_game_over():
            print('game_over')
            break

h2h3

r n b q k b n r
p p p . . p p p
. . . . p . . .
. . . p . . . .
. . . P . . . .
. . . . B . . P
P P P . P P P .
R N . Q K B N R
